# Bert experimenty

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from typing import Dict, List
import transformers
from transformers import AutoModel, BertTokenizerFast,BertTokenizer, DistilBertTokenizerFast
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM

In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install transformers==4.30

## Načítanie a príprava dát

In [ ]:
dataset = pd.read_csv('../Data/final_dataset_3.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_text'], dataset['author_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

## Bert

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
x_train = list(X_train)
x_val = list(X_valid)
x_test = list(X_test)

y_train = list(y_train)
y_val = list(y_valid)
y_test = list(y_test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_valid)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
labels_count = len(dataset['author_id'].unique())
print(labels_count)

20


## Experiment 1 - learning_rate = 5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=5e-5,             #default
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.903996,0.096154,0.089808,0.082817,0.059489
2,No log,2.251390,0.455128,0.362781,0.394313,0.319627
3,No log,1.565826,0.596154,0.432527,0.586250,0.477056
4,No log,1.008685,0.717949,0.736071,0.729470,0.688056
5,No log,0.840192,0.794872,0.809317,0.790575,0.785518
6,1.671600,0.760379,0.769231,0.785084,0.771035,0.754625
7,1.671600,0.655085,0.814103,0.821806,0.799127,0.797172
8,1.671600,0.560627,0.884615,0.887996,0.867222,0.867706
9,1.671600,0.589349,0.852564,0.857901,0.846052,0.844222
10,1.671600,0.602404,0.858974,0.863796,0.848909,0.848755


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=1056, training_loss=0.8285907453038928, metrics={'train_runtime': 764.7734, 'train_samples_per_second': 22.03, 'train_steps_per_second': 1.381, 'total_flos': 4433611481874432.0, 'train_loss': 0.8285907453038928, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 7.3339844e-01,  1.8127441e-01, -4.1040039e-01, ...,
        -1.6035156e+00,  5.9843750e+00, -4.7607422e-01],
       [-1.1501312e-03,  1.7041016e+00,  5.9687500e+00, ...,
        -7.5927734e-01, -8.6279297e-01, -7.6123047e-01],
       [ 1.5878906e+00,  6.1328125e+00,  1.7773438e+00, ...,
        -1.2080078e+00, -5.3564453e-01, -1.3994141e+00],
       ...,
       [-9.4042969e-01, -7.1289062e-01, -8.3056641e-01, ...,
        -5.9667969e-01,  5.4882812e+00, -2.8247070e-01],
       [-5.4248047e-01, -2.8271484e-01, -4.8901367e-01, ...,
        -5.4052734e-01,  7.2734375e+00,  5.3955078e-01],
       [-2.5558472e-02, -2.2753906e+00, -1.9042969e+00, ...,
         1.5698242e-01,  1.4482422e+00,  1.2402344e+00]], dtype=float32), label_ids=array([18,  1,  2, 18, 15,  9,  1, 19,  6, 15, 15, 18,  1, 12, 14,  8,  4,
       19,  9, 16, 17, 16,  7, 13,  0,  7, 15, 15, 11,  4, 17, 12,  5, 18,
       12,  7, 16,  1, 17, 19,  2,  8,  4,  2, 19,  7,  7,  7,  8,  5,  7,

In [ ]:
#!pip install transformers==4.30
# riesi error
#Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install transformers==4.30

## Experiment 4 - learning_rate = 8e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=8e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.783504,0.160256,0.027583,0.120536,0.044068
2,No log,1.918989,0.448718,0.328553,0.387898,0.302768
3,No log,1.373838,0.576923,0.542281,0.581370,0.503306
4,No log,1.016014,0.685897,0.687014,0.698191,0.658744
5,No log,0.820310,0.756410,0.807272,0.754542,0.744611
6,1.439600,0.771604,0.788462,0.824572,0.787599,0.778141
7,1.439600,1.165571,0.737179,0.780369,0.728794,0.724227
8,1.439600,0.762248,0.858974,0.852083,0.851111,0.842604
9,1.439600,0.708223,0.858974,0.878740,0.855278,0.855848
10,1.439600,0.643722,0.858974,0.869187,0.864444,0.852621


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=1056, training_loss=0.7077338922430169, metrics={'train_runtime': 771.5513, 'train_samples_per_second': 21.837, 'train_steps_per_second': 1.369, 'total_flos': 4433611481874432.0, 'train_loss': 0.7077338922430169, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-0.8486328 , -0.5727539 , -0.79785156, ..., -0.27392578,
         7.84375   , -0.43945312],
       [-0.1204834 ,  0.4716797 ,  6.078125  , ..., -0.09143066,
         1.8408203 , -0.6098633 ],
       [ 1.7373047 ,  5.6914062 ,  5.75      , ..., -0.30737305,
        -0.72558594, -1.4921875 ],
       ...,
       [-1.1855469 , -0.31469727, -0.63671875, ..., -0.5161133 ,
         7.6992188 , -0.49145508],
       [-0.99072266, -0.34277344, -0.60302734, ..., -0.4033203 ,
         7.9023438 , -0.3190918 ],
       [ 1.1220703 , -1.3085938 , -0.6503906 , ...,  0.31469727,
         0.25683594,  1.546875  ]], dtype=float32), label_ids=array([18,  1,  2, 18, 15,  9,  1, 19,  6, 15, 15, 18,  1, 12, 14,  8,  4,
       19,  9, 16, 17, 16,  7, 13,  0,  7, 15, 15, 11,  4, 17, 12,  5, 18,
       12,  7, 16,  1, 17, 19,  2,  8,  4,  2, 19,  7,  7,  7,  8,  5,  7,
       11,  8,  5, 10,  9,  9, 19, 19,  8,  2, 12, 12, 19,  6, 12, 15,  0,
       18, 12,  6,  8, 12, 16,  